## 1. Environment Setup

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

BATCH_SIZE = 64
IMAGE_SIZE = (224, 224)
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print(e)

precision = 'mixed_float16'
policy = tf.keras.mixed_precision.Policy(precision)
tf.keras.mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce GTX 1650, compute capability 7.5


In [25]:
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'
Inception_V3 = 'https://tfhub.dev/google/imagenet/inception_v3/classification/5'
MobileNet_V3_Large = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5'
Inception_ResNet_V2 = 'https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5'
EfficientNet_B0 = 'https://tfhub.dev/google/efficientnet/b0/classification/1'

In [6]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy') > 0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

def lr_schedule(epoch):
  initial_lr = 1e-3
  new_lr = initial_lr * 0.9 ** epoch

  return new_lr

callbacks = myCallback()
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [15]:
def preprocess_image(image):
  image = tf.image.random_flip_left_right(image)
  image = tf.cast(image, tf.float32) / 255.0
  return image

def get_label(file_path, class_names):
  parts = tf.strings.split(file_path, os.sep)
  label = parts[-2] == class_names
  return label

def load_image_and_label(file_path, class_names):
  label = get_label(file_path, class_names)
  image = tf.io.read_file(file_path)
  image = tf.image.decode_jpeg(image, channels=3)
  return image, label

def prepare_dataset(directory, class_names, batch_size, training=True):
  dataset = tf.data.Dataset.list_files(os.path.join(directory, '*/*'))
  dataset = dataset.map(lambda x: load_image_and_label(x, class_names), num_parallel_calls=AUTOTUNE)
  dataset = dataset.map(lambda x, y: (preprocess_image(x), y), num_parallel_calls=AUTOTUNE)
  if training:
    dataset = dataset.shuffle(buffer_size=1000)

  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=AUTOTUNE)
  return dataset

def create_dataset(train_dir, test_dir, batch_size=BATCH_SIZE):
  class_names = tf.io.gfile.listdir(train_dir)
  total_class = len(class_names)
  class_names = tf.constant(class_names)

  train_dataset = prepare_dataset(train_dir, class_names, batch_size, training=True)
  test_dataset = prepare_dataset(test_dir, class_names, batch_size, training=False)

  return train_dataset, test_dataset, total_class
  

In [19]:
def create_model(pre_trained_model, total_class):
  feature_extractor_layer = hub.KerasLayer(pre_trained_model, input_shape=(224, 224, 3), trainable=False, dtype=tf.float32)
  model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), dtype=tf.float32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), dtype=tf.float32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(total_class, activation='softmax')
  ])

  model.summary()

  model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
  )

  return model

def fit_model(model, train_generator, test_generator, epochs, callbacks = []):
  history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=epochs,
    callbacks=callbacks
  )

  return history

def save_model(model, pre_trained_model, type):
  model.save('model_MobileNet_Makanan_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

In [44]:
def get_average_metric(model_history, metric_name, epochs = 10):
  metric = model_history.history[metric_name]
  avg_metric = np.mean(metric[-epochs:])
  return avg_metric

def predict(model, test_dataset: tf.data.Dataset, total_class):
  wrong_images = []
  wrong_labels = []
  wrong_predictions = []
  wrong_actual = np.zeros(total_class)
  total_class_data = np.zeros(total_class)
  test_dataset = test_dataset.shuffle(buffer_size=1000)
  test_dataset = test_dataset.take(int(len(test_dataset) * 0.6))
  for images, labels in test_dataset:
    predictions = model.predict(images)
    for i in range(len(predictions)):
      if np.argmax(predictions[i]) != np.argmax(labels[i]):
        wrong_predictions.append(predictions[i])
        wrong_images.append(images[i])
        wrong_labels.append(labels[i])
        wrong_actual[np.argmax(labels[i])] += 1
      total_class_data[np.argmax(labels[i])] += 1

def show_missclassified_images(wrong_images, wrong_labels, wrong_predictions, class_names, test_generator):
  total_row = int(np.ceil(len(wrong_predictions) / 5))
  fig = plt.figure(figsize=(20, total_row * 5))
  for i in range(total_row):
    remainder = len(wrong_predictions) - i * 5
    total = remainder if remainder < 5 else 5
    for j in range(total):
      idx = i * 5 + j
      ax = fig.add_subplot(total_row, 5, idx + 1)
      ax.imshow(wrong_images[idx])
      ax.set_title('Prediction: ' + class_names[np.argmax(wrong_predictions[idx])] + ' ' + str(round(np.sort(wrong_predictions[idx])[-1] * 100, 2)) +
                  '\nPrediction2: ' + class_names[np.argsort(wrong_predictions[idx])[-2]] + ' ' + str(round(np.sort(wrong_predictions[idx])[-2] * 100, 2)) +
                    '\nPrediction3: ' + class_names[np.argsort(wrong_predictions[idx])[-3]] + ' ' + str(round(np.sort(wrong_predictions[idx])[-3] * 100, 2)) +
                  '\nActual: ' + class_names[np.argmax(wrong_labels[idx])] + 
                  '\nFilename: ' + test_generator.filenames[idx])
      ax.axis('off')

def plot_graphs(model_history, metric):
  plt.plot(model_history.history[metric])
  plt.plot(model_history.history['val_'+metric])
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric], loc='upper right')
  plt.title('Model ' + metric)

## 2. Model Jajanan Pasar Creation

In [ ]:
pre_trained_model = MobileNet_V3_Large
train_path = './dataset/jajanan/train/'
test_path = './dataset/jajanan/test/'

train_gen, test_gen, total_class = create_dataset(train_path, test_path)

model = create_model(pre_trained_model, total_class)
model_history = fit_model(model, train_gen, test_gen, 50, [callbacks, lr_scheduler, early_stopping])

In [ ]:
class_names = os.listdir(train_path)

wrong_pred, wrong_images, wrong_labels, wrong_actual_percentage, total_class_data = predict(model, test_gen, total_class)
print('Accuracy: ', (test_gen.n - len(wrong_pred)) / test_gen.n * 100, '%')
print('Wrong Prediction: ')
for i in range(total_class):
  print(class_names[i], ': ', wrong_actual_percentage[i], '/', total_class_data[i])

In [ ]:
show_missclassified_images(wrong_images, wrong_labels, wrong_pred, class_names, test_gen)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plot_graphs(model, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(model, 'loss')
plt.show()

## 3. Model Makanan Tradisional Creation

In [27]:
pre_trained_model = EfficientNet_B0
train_path = './dataset/makanan/train/'
test_path = './dataset/makanan/test/'

train_gen, test_gen, total_class = create_dataset(train_path, test_path)

In [28]:

model = create_model(pre_trained_model, total_class)
model_history = fit_model(model, train_gen, test_gen, 50, [callbacks, lr_scheduler, early_stopping])

save_model(model, 'EfficientNet_B0', 'Makanan')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5 (KerasLayer)  (None, 1000)              5330564   
                                                                 
 flatten_5 (Flatten)         (None, 1000)              0         
                                                                 
 dense_12 (Dense)            (None, 1024)              1025024   
                                                                 
 dropout_7 (Dropout)         (None, 1024)              0         
                                                                 
 dense_13 (Dense)            (None, 512)               524800    
                                                                 
 dropout_8 (Dropout)         (None, 512)               0         
                                                                 
 dense_14 (Dense)            (None, 31)               

In [ ]:
class_names = os.listdir(train_path)

wrong_pred, wrong_images, wrong_labels, wrong_actual_percentage, total_class_data = predict(model, test_gen, total_class)
print('Accuracy: ', (test_gen.n - len(wrong_pred)) / test_gen.n * 100, '%')
print('Wrong Prediction: ')
for i in range(total_class):
  print(class_names[i], ': ', wrong_actual_percentage[i], '/', total_class_data[i])

In [ ]:
show_missclassified_images(wrong_images, wrong_labels, wrong_pred, class_names, test_gen)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plot_graphs(model, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(model, 'loss')
plt.show()